In [47]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
recipes = pd.read_csv('C:/Users/ARAYN/OneDrive/Desktop/recipe_generator/data/recipes.csv')

In [49]:
recipes

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...,...,...,...,...,...
2231137,2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
2231138,2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
2231139,2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
2231140,2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."


In [50]:
recipes.columns

Index(['Unnamed: 0', 'title', 'ingredients', 'directions', 'link', 'source',
       'NER'],
      dtype='object')

In [51]:
recipes.drop(['Unnamed: 0'], axis=1, inplace=True)

In [52]:
recipes.head()

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [53]:
#Function to clean NER entries
def preprocess_ner(ner_list):
    return [re.sub(r'[^a-zA-Z\s]', '', ner.lower()).strip() for ner in ner_list]

In [54]:
recipes['NER']=recipes['NER'].apply(eval).apply(preprocess_ner)


In [55]:
#further simplification of ingredients
simplify_dict ={
    'bite size shredded rice biscuits': 'biscuits'
}


def singularize_word (word):
    singular = word
    #Using WordNet to get singular form if available
    for syn in wordnet.synsets(word):
        if syn.name().split(' ')[0]==word:
            
            singular = syn.lemmas()[0].name() 
            break
    return singular


#Function to simplify ingredients
def simplify_ingredients(ingredient_list):
    simplified = []
    for ing in ingredient_list:
        if not ing.strip():  #Skip empty strings
            continue
        ing = ing.lower()   #Convert to lowercase
        for key,value in simplify_dict.items():
            if key in ing:
                ing = value
        simplified.append(ing)
    return simplified

recipes['simplified_NER']=recipes['NER'].apply(simplify_ingredients)

In [56]:
recipes

,title,ingredients,directions,link,source,NER,simplified_NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[brown sugar, milk, vanilla, nuts, butter, bit...","[brown sugar, milk, vanilla, nuts, butter, bis..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[beef, chicken breasts, cream of mushroom soup...","[beef, chicken breasts, cream of mushroom soup..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[frozen corn, cream cheese, butter, garlic pow...","[frozen corn, cream cheese, butter, garlic pow..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[chicken, chicken gravy, cream of mushroom sou...","[chicken, chicken gravy, cream of mushroom sou..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[peanut butter, graham cracker crumbs, butter,...","[peanut butter, graham cracker crumbs, butter,..."
...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[chocolate hazelnut spread, tortillas, butter,...","[chocolate hazelnut spread, tortillas, butter,..."
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[eggs, paprika, salt, choice, miracle whip, re...","[eggs, paprika, salt, choice, miracle whip, re..."
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[radish, sesame oil, white sesame seeds, salt,...","[radish, sesame oil, white sesame seeds, salt,..."
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[apple cider, sugar, kosher salt, bay leaves, ...","[apple cider, sugar, kosher salt, bay leaves, ..."


In [57]:
#Combine all ingredients into a single string for each recipe
recipes['ingredient_string'] =recipes['simplified_NER'].apply(lambda x: ' '.join(x))

#Apply TF-IDF vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(recipes['ingredient_string'])

#Show the shape of the resulting matric
print(tfidf_matrix.shape)

(2231142, 35499)


In [58]:
def vectorize_user_input(user_input, vectorizer):
    #Clean and simplify user input 
    simplified_user_input = simplify_ingredients(user_input)
    user_input_str =' '.join(simplified_user_input)

    #Vectorize the user's input
    user_input_vector = vectorizer.transform([user_input_str])

    return user_input_vector

In [60]:
def recommend_recipes(user_input):
    user_input_vector = vectorize_user_input(user_input, tfidf)
    
    #Calculate cosine similarity
    cosine_similarities = cosine_similarity(user_input_vector, tfidf_matrix)
    
    #Get he indices of the top 5 most similar recipes
    top_indices = cosine_similarities.argsort()[0][-10:][::-1]
    
    #Display the most similar recipes
    for idx in top_indices:
        # print(idx
        recipe = recipes.iloc[idx]# Join directions list into a string
        print(f"Recipe: {recipe['title']} - Similarity: {cosine_similarities[0][idx]:.4f}")
        print(f"Ingredients: {recipe['NER']}")
        # print(f"Directions: {directions_str}")
        print("-" * 50)

    # Create a list of the top recipes
    recommended_recipes = recipes.iloc[top_indices]
    print("Recommended Recipes:")
    return(recommended_recipes[['title','ingredients','directions']])


In [61]:
user_input = ['chicken','egg','tomato','lettuce']
print(recommend_recipes(user_input))

Recipe: Tomato Slaw - Similarity: 0.7096
Ingredients: ['lettuce', 'tomato', 'mayonnaise']
--------------------------------------------------
Recipe: Tomato And Egg Salad - Similarity: 0.6179
Ingredients: ['tomato', 'egg']
--------------------------------------------------
Recipe: Karaage (Japanese Fried Chicken) in Cups - Similarity: 0.6142
Ingredients: ['chicken', 'tomato']
--------------------------------------------------
Recipe: CHEEZ WHIZ Process Cheese Spread Toasted Tomato Sandwich - Similarity: 0.6037
Ingredients: ['bread', 'tomato', 'lettuce leaves']
--------------------------------------------------
Recipe: Christmas Cocktail Sandwiches - Similarity: 0.5921
Ingredients: ['bread', 'margarine', 'each lettuce', 'egg']
--------------------------------------------------
Recipe: Salad Nicoise - Similarity: 0.5911
Ingredients: ['tomato', 'lettuce', 'cucumber', 'onion', 'eggs', 'much', 'much', 'much']
--------------------------------------------------
Recipe: How to Tear Lettuce to M

In [62]:
from gensim.models import Word2Vec

In [63]:
model = Word2Vec(sentences=recipes['simplified_NER'], vector_size=50, window=5, min_count=1, workers=4)

In [65]:
def get_average_embedding(ingredients, model):
        vectors = [model.wv[ingredient] for ingredient in ingredients if ingredient in model.wv]
        return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

def recommend_recipes_word2vec(user_input):
    
    """
    Recommend the top 10 recipes based on Word2Vec ingredient similarity.

    Parameters:
        user_input (list): List of ingredients input by the user.
        recipes (list of lists): A list of recipes, where each recipe is a list of ingredients.
        word2vec_model (gensim.models.Word2Vec): Pre-trained Word2Vec model.

    Returns:
        list: Top 10 recommended recipes with their similarity scores.
    """

    user_embedding = get_average_embedding(user_input, model)
    recipe_embedding = np.array(recipes['embedding'].tolist())

    similarities = cosine_similarity([user_embedding], recipe_embedding)

    
    top_indices = similarities[0].argsort()[-10:][::-1]
    for idx in top_indices:
        recipe = recipes.iloc[idx]
        ingredients_str = ', '.join(recipe['ingredients'])  # Join ingredients list into a string
        directions_str = ' '.join(recipe['directions'])  # Join directions list into a string
        print(f"Recipe: {recipe['title']} - Similarity: {similarities[0][idx]:.4f}")
        print(f"Ingredients: {recipe['simplified_NER']}")
        # print(f"Directions: {directions_str}")
        print("-" * 50)
    top_recipes = recipes.iloc[top_indices]
    return top_recipes[['title','ingredients','directions']]

In [66]:
recipes['embedding'] = recipes['simplified_NER'].apply(
    lambda ingredient: get_average_embedding(ingredient, model)
)

In [67]:
recipes

,title,ingredients,directions,link,source,NER,simplified_NER,ingredient_string,embedding
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[brown sugar, milk, vanilla, nuts, butter, bit...","[brown sugar, milk, vanilla, nuts, butter, bis...",brown sugar milk vanilla nuts butter biscuits,"[0.946958, -0.18480472, -0.7164858, -1.976132,..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[beef, chicken breasts, cream of mushroom soup...","[beef, chicken breasts, cream of mushroom soup...",beef chicken breasts cream of mushroom soup so...,"[-2.4311407, 0.39234567, 1.662856, -1.0655999,..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[frozen corn, cream cheese, butter, garlic pow...","[frozen corn, cream cheese, butter, garlic pow...",frozen corn cream cheese butter garlic powder ...,"[-0.2807318, 1.0975028, -0.070980035, -2.48269..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[chicken, chicken gravy, cream of mushroom sou...","[chicken, chicken gravy, cream of mushroom sou...",chicken chicken gravy cream of mushroom soup s...,"[-2.1025338, 0.5496192, 3.1503298, -0.8616563,..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[peanut butter, graham cracker crumbs, butter,...","[peanut butter, graham cracker crumbs, butter,...",peanut butter graham cracker crumbs butter pow...,"[1.6544046, -0.067045495, -1.704285, -1.374880..."
...,...,...,...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[chocolate hazelnut spread, tortillas, butter,...","[chocolate hazelnut spread, tortillas, butter,...",chocolate hazelnut spread tortillas butter mar...,"[0.24092205, -0.99941623, -0.41935387, -0.7675..."
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[eggs, paprika, salt, choice, miracle whip, re...","[eggs, paprika, salt, choice, miracle whip, re...",eggs paprika salt choice miracle whip relish,"[-0.28726104, 0.5797631, -0.7770917, -2.01112,..."
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[radish, sesame oil, white sesame seeds, salt,...","[radish, sesame oil, white sesame seeds, salt,...",radish sesame oil white sesame seeds salt soy ...,"[-0.87837696, 2.491654, -3.1197596, -1.5681553..."
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[apple cider, sugar, kosher salt, bay leaves, ...","[apple cider, sugar, kosher salt, bay leaves, ...",apple cider sugar kosher salt bay leaves arbol...,"[-0.8054368, 0.10362331, -0.25036475, -2.61943..."


In [68]:
print(recommend_recipes_word2vec(user_input))

Recipe: Salade de gesier (gizzard salad) - Similarity: 0.9277
Ingredients: ['chicken', 'pinenuts', 'lettuce', 'tomatoes', 'dressing', 'egg']
--------------------------------------------------
Recipe: Deluxe Chef Salad - Similarity: 0.9107
Ingredients: ['head', 'chicken', 'cheddar cheese', 'tomato', 'eggs', 'wishbone deluxe']
--------------------------------------------------
Recipe: California Club Salad - Similarity: 0.8912
Ingredients: ['shredded lettuce', 'chicken', 'eggs', 'bacon', 'tomatoes', 'avocado']
--------------------------------------------------
Recipe: Ranch Tuna Cobb Salad - Similarity: 0.8884
Ingredients: ['green leaf', 'tomatoes', 'chicken', 'avocado', 'eggs', 'green onions', 'bacon', 'wish bone ranch dress']
--------------------------------------------------
Recipe: Mini Taco Hand Pies - Similarity: 0.8876
Ingredients: ['quarters', 'chicken', 'tomato', 'egg', 'sour cream']
--------------------------------------------------
Recipe: The Best Open-Faced Egg Sandwich - Si

In [69]:
model.save('recipe_recommendation.model')

In [71]:
from gensim.models import Word2Vec
loaded_model = Word2Vec.load("recipe_recommendation.model")

In [72]:
print(loaded_model.wv.most_similar('chicken'))

[('chicken breasts', 0.9129457473754883), ('chicken breast', 0.8656049370765686), ('chicken thighs', 0.8635833263397217), ('skinless', 0.8559930920600891), ('chickens', 0.8107917308807373), ('chicken breast halves', 0.8050429821014404), ('fryer', 0.7762293219566345), ('turkey', 0.7715348601341248), ('chicken meat', 0.7658758759498596), ('boneless chicken', 0.7543060183525085)]
